In [1]:
import os
print("JAVA_HOME =", os.environ.get("JAVA_HOME"))

JAVA_HOME = None


#### Verified which Python my notebook is using

In [2]:
import sys
print(sys.executable)

/Users/pernebayarailym/anaconda3/envs/myenv/bin/python


#### Ensured PySpark installs in the same Python environment my notebook is using

In [3]:
import sys
!{sys.executable} -m pip install pyspark

In [4]:
import pyspark
print(pyspark.__version__)

4.0.1


##### Used these commands to install apache-spark because it couldn't find from homebrew , so I manually set the SPARK_HOME path using the path from the command below:

1) brew install apache-spark
2) brew --prefix apache-spark



In [5]:
import os
os.environ['JAVA_HOME'] = '/opt/homebrew/opt/openjdk@17/libexec/openjdk.jdk/Contents/Home'

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MyApp_cleaning") \
    .getOrCreate()

print(spark.version)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/09/30 13:37:35 WARN Utils: Your hostname, MacBook-Air--Arailym.local, resolves to a loopback address: 127.0.0.1; using 172.20.10.2 instead (on interface en0)
25/09/30 13:37:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/30 13:37:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


4.0.1


In [6]:
# Import necessary PySpark modules
from pyspark.sql import SparkSession #is the entry point to PySpark
from pyspark.sql import functions as F 
from pyspark.sql import types as T

In [7]:
# Read CSV files
csv_path = '/Users/pernebayarailym/Documents/Portfolio_Projects_AP/Simplon_DE_Projects/Python_Projects/Project_Pyspark_DBT/notebooks/data/ventes.csv'
#'/Users/pernebayarailym/Documents/Portfolio_Projects_AP/Simplon_DE_Projects/Python_Projects/Project_Pyspark_DBT/data/ventes.csv'
df = spark.read \
    .option("header", True) \
    .option("inferSchema", False) \
    .csv(csv_path)

df.printSchema()
df.show(5)

root
 |-- id_transaction: string (nullable = true)
 |-- client_nom: string (nullable = true)
 |-- client_age: string (nullable = true)
 |-- client_ville: string (nullable = true)
 |-- produit_nom: string (nullable = true)
 |-- produit_categorie: string (nullable = true)
 |-- produit_marque: string (nullable = true)
 |-- prix_catalogue: string (nullable = true)
 |-- magasin_nom: string (nullable = true)
 |-- magasin_type: string (nullable = true)
 |-- magasin_region: string (nullable = true)
 |-- date: string (nullable = true)
 |-- quantite: string (nullable = true)
 |-- montant_total: string (nullable = true)

+--------------+----------+----------+------------+----------------+-----------------+--------------+--------------+--------------+------------+--------------------+----------+--------+-------------+
|id_transaction|client_nom|client_age|client_ville|     produit_nom|produit_categorie|produit_marque|prix_catalogue|   magasin_nom|magasin_type|      magasin_region|      date|quanti

In [8]:
#1 task. Normalize string columns (lowercase + remove extra spaces)
#detecting which columns are string
string_cols = [f.name for f in df.schema.fields if isinstance(f.dataType, T.StringType)]

# for each string column, convert to lowercase and trim spaces
for c in string_cols:
    df = df.withColumn(c, F.lower(F.trim(F.col(c))))
df.show(5)

+--------------+----------+----------+------------+----------------+-----------------+--------------+--------------+--------------+------------+--------------------+----------+--------+-------------+
|id_transaction|client_nom|client_age|client_ville|     produit_nom|produit_categorie|produit_marque|prix_catalogue|   magasin_nom|magasin_type|      magasin_region|      date|quantite|montant_total|
+--------------+----------+----------+------------+----------------+-----------------+--------------+--------------+--------------+------------+--------------------+----------+--------+-------------+
|             1|     alice|        25|       paris|      ordinateur|     informatique|          dell|           800| boutique lyon|    physique|auvergne-rhône-alpes|2023-03-12|       2|         NULL|
|             2|       bob|        34|        lyon|      smartphone|       téléphonie|         apple|          1200| boutique lyon|    physique|auvergne-rhône-alpes|2023-01-27|       5|         NULL|


In [9]:
#2 task. Replace empty strings with None (NULL) to detect missing values
for c in df.columns:
    df = df.withColumn(c, F.when(F.col(c) == "", None).otherwise(F.col(c)))

#  Drop rows with missing values, except 'montant_total because it will be recalculated later and for now it has NULL values everywhere'
cols_to_check = [c for c in df.columns if c != "montant_total"]
df = df.dropna(subset=cols_to_check, how="any") #drop rows with any NULL values
df.show(10)

+--------------+----------+----------+------------+----------------+-----------------+--------------+--------------+--------------+------------+--------------------+----------+--------+-------------+
|id_transaction|client_nom|client_age|client_ville|     produit_nom|produit_categorie|produit_marque|prix_catalogue|   magasin_nom|magasin_type|      magasin_region|      date|quantite|montant_total|
+--------------+----------+----------+------------+----------------+-----------------+--------------+--------------+--------------+------------+--------------------+----------+--------+-------------+
|             1|     alice|        25|       paris|      ordinateur|     informatique|          dell|           800| boutique lyon|    physique|auvergne-rhône-alpes|2023-03-12|       2|         NULL|
|             2|       bob|        34|        lyon|      smartphone|       téléphonie|         apple|          1200| boutique lyon|    physique|auvergne-rhône-alpes|2023-01-27|       5|         NULL|


In [10]:
#3 task. Handle purchase date 
#parse the purchase date column into a proper date type
df = df.withColumn(
    "purchase_date", 
    F.coalesce(
        F.to_date(F.col("date"), "yyyy-MM-dd"),
        F.to_date(F.col("date"), "dd/MM/yyyy"),
        F.to_date(F.col("date"), "MM/dd/yyyy"),
        F.to_date(F.col("date"), "yyyy/MM/dd"),

    )
)
df.show(10)
#coalesce() tries several formats and picks the first that works


+--------------+----------+----------+------------+----------------+-----------------+--------------+--------------+--------------+------------+--------------------+----------+--------+-------------+-------------+
|id_transaction|client_nom|client_age|client_ville|     produit_nom|produit_categorie|produit_marque|prix_catalogue|   magasin_nom|magasin_type|      magasin_region|      date|quantite|montant_total|purchase_date|
+--------------+----------+----------+------------+----------------+-----------------+--------------+--------------+--------------+------------+--------------------+----------+--------+-------------+-------------+
|             1|     alice|        25|       paris|      ordinateur|     informatique|          dell|           800| boutique lyon|    physique|auvergne-rhône-alpes|2023-03-12|       2|         NULL|   2023-03-12|
|             2|       bob|        34|        lyon|      smartphone|       téléphonie|         apple|          1200| boutique lyon|    physique|

In [11]:
#3 task. Remove extravagant dates
df =df.filter(
    (F.col("purchase_date") >= F.to_date(F.lit("2023-01-01"))) &
    (F.col("purchase_date") <= F.current_date())
)

df.show(5)

+--------------+----------+----------+------------+----------------+-----------------+--------------+--------------+--------------+------------+--------------------+----------+--------+-------------+-------------+
|id_transaction|client_nom|client_age|client_ville|     produit_nom|produit_categorie|produit_marque|prix_catalogue|   magasin_nom|magasin_type|      magasin_region|      date|quantite|montant_total|purchase_date|
+--------------+----------+----------+------------+----------------+-----------------+--------------+--------------+--------------+------------+--------------------+----------+--------+-------------+-------------+
|             1|     alice|        25|       paris|      ordinateur|     informatique|          dell|           800| boutique lyon|    physique|auvergne-rhône-alpes|2023-03-12|       2|         NULL|   2023-03-12|
|             2|       bob|        34|        lyon|      smartphone|       téléphonie|         apple|          1200| boutique lyon|    physique|

In [12]:
#4 task. handle numeric anomalies (checking the columns like age, quantite, prix_unitaire are numbers not strings
numeric_cols = {
    "client_age": T.IntegerType(),
    "quantite": T.IntegerType(),
    "prix_catalogue": T.DoubleType(),
    #"montant_total": T.DoubleType()
}

for col_name, col_type in numeric_cols.items():
    if col_name in df.columns: 
        df = df.withColumn(col_name, F.col(col_name).cast(col_type))


#cast() converts a column to a given type , if
#the value can't be converted then it becomes NULL like if it's "abc"

df.show(5)

+--------------+----------+----------+------------+----------------+-----------------+--------------+--------------+--------------+------------+--------------------+----------+--------+-------------+-------------+
|id_transaction|client_nom|client_age|client_ville|     produit_nom|produit_categorie|produit_marque|prix_catalogue|   magasin_nom|magasin_type|      magasin_region|      date|quantite|montant_total|purchase_date|
+--------------+----------+----------+------------+----------------+-----------------+--------------+--------------+--------------+------------+--------------------+----------+--------+-------------+-------------+
|             1|     alice|        25|       paris|      ordinateur|     informatique|          dell|         800.0| boutique lyon|    physique|auvergne-rhône-alpes|2023-03-12|       2|         NULL|   2023-03-12|
|             2|       bob|        34|        lyon|      smartphone|       téléphonie|         apple|        1200.0| boutique lyon|    physique|

In [13]:
#drop rows where numeric columns could not be converted
for col_name in numeric_cols.keys():
    if col_name in df.columns:
        df = df.filter(F.col(col_name).isNotNull())

df.show(5)

+--------------+----------+----------+------------+----------------+-----------------+--------------+--------------+--------------+------------+--------------------+----------+--------+-------------+-------------+
|id_transaction|client_nom|client_age|client_ville|     produit_nom|produit_categorie|produit_marque|prix_catalogue|   magasin_nom|magasin_type|      magasin_region|      date|quantite|montant_total|purchase_date|
+--------------+----------+----------+------------+----------------+-----------------+--------------+--------------+--------------+------------+--------------------+----------+--------+-------------+-------------+
|             1|     alice|        25|       paris|      ordinateur|     informatique|          dell|         800.0| boutique lyon|    physique|auvergne-rhône-alpes|2023-03-12|       2|         NULL|   2023-03-12|
|             2|       bob|        34|        lyon|      smartphone|       téléphonie|         apple|        1200.0| boutique lyon|    physique|

In [14]:
#5 task. Handle negative ages
if "client_age" in df.columns:
    df =df.withColumn("client_age", F.abs(F.col("client_age")))
df.show(5)

+--------------+----------+----------+------------+----------------+-----------------+--------------+--------------+--------------+------------+--------------------+----------+--------+-------------+-------------+
|id_transaction|client_nom|client_age|client_ville|     produit_nom|produit_categorie|produit_marque|prix_catalogue|   magasin_nom|magasin_type|      magasin_region|      date|quantite|montant_total|purchase_date|
+--------------+----------+----------+------------+----------------+-----------------+--------------+--------------+--------------+------------+--------------------+----------+--------+-------------+-------------+
|             1|     alice|        25|       paris|      ordinateur|     informatique|          dell|         800.0| boutique lyon|    physique|auvergne-rhône-alpes|2023-03-12|       2|         NULL|   2023-03-12|
|             2|       bob|        34|        lyon|      smartphone|       téléphonie|         apple|        1200.0| boutique lyon|    physique|

In [15]:
#6 task.Calculate total amount per sale
# drop the old useless NULL column if it exists
# add the computed column

if "montant_total" in df.columns:
    df = df.drop("montant_total")   # remove the placeholder column

# create new montant_total column
df = df.withColumn("montant_total", F.col("quantite") * F.col("prix_catalogue"))

df.show(10)


+--------------+----------+----------+------------+----------------+-----------------+--------------+--------------+--------------+------------+--------------------+----------+--------+-------------+-------------+
|id_transaction|client_nom|client_age|client_ville|     produit_nom|produit_categorie|produit_marque|prix_catalogue|   magasin_nom|magasin_type|      magasin_region|      date|quantite|purchase_date|montant_total|
+--------------+----------+----------+------------+----------------+-----------------+--------------+--------------+--------------+------------+--------------------+----------+--------+-------------+-------------+
|             1|     alice|        25|       paris|      ordinateur|     informatique|          dell|         800.0| boutique lyon|    physique|auvergne-rhône-alpes|2023-03-12|       2|   2023-03-12|       1600.0|
|             2|       bob|        34|        lyon|      smartphone|       téléphonie|         apple|        1200.0| boutique lyon|    physique|

In [16]:
import sys
!{sys.executable} -m pip install pandas


In [17]:
import os

# Always resolve relative to the project folder
project_root = os.getcwd()   # prints current working directory
print("Current working dir:", project_root)

output_path = os.path.join(project_root, "data", "ventes_clean.csv")

df.toPandas().to_csv(output_path, index=False)
print("Saved to:", output_path)


Current working dir: /Users/pernebayarailym/Documents/Portfolio_Projects_AP/Simplon_DE_Projects/Python_Projects/Project_Pyspark_DBT/notebooks
Saved to: /Users/pernebayarailym/Documents/Portfolio_Projects_AP/Simplon_DE_Projects/Python_Projects/Project_Pyspark_DBT/notebooks/data/ventes_clean.csv


In [18]:
import sys
!{sys.executable} -m pip install duckdb

In [ ]:
import duckdb

# Connect to  DuckDB database file
con = duckdb.connect('/Users/pernebayarailym/Documents/Portfolio_Projects_AP/Simplon_DE_Projects/Python_Projects/Project_Pyspark_DBT/duckdb/ventes_clean.duckdb')

# Create a new table from the cleaned CSV
con.execute("""
    CREATE OR REPLACE TABLE ventes_clean AS
    SELECT * FROM 'data/ventes_clean.csv'
""")

# Verify by selecting a few rows
result = con.execute("SELECT * FROM ventes_clean LIMIT 5").fetchdf()
result


,id_transaction,client_nom,client_age,client_ville,produit_nom,produit_categorie,produit_marque,prix_catalogue,magasin_nom,magasin_type,magasin_region,date,quantite,purchase_date,montant_total
0,1,alice,25,paris,ordinateur,informatique,dell,800.0,boutique lyon,physique,auvergne-rhône-alpes,2023-03-12,2,2023-03-12,1600.0
1,2,bob,34,lyon,smartphone,téléphonie,apple,1200.0,boutique lyon,physique,auvergne-rhône-alpes,2023-01-27,5,2023-01-27,6000.0
2,3,alice,25,paris,montre connectée,accessoires,garmin,300.0,e-shop,en ligne,national,2023-01-09,1,2023-01-09,300.0
3,4,alice,25,paris,smartphone,téléphonie,apple,1200.0,boutique paris,physique,île-de-france,2023-05-10,5,2023-05-10,6000.0
4,5,alice,25,paris,montre connectée,accessoires,garmin,300.0,boutique paris,physique,île-de-france,2023-06-16,5,2023-06-16,1500.0


25/10/01 02:21:21 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 997124 ms exceeds timeout 120000 ms
25/10/01 02:21:21 WARN SparkContext: Killing executors is not supported by current scheduler.
25/10/01 02:21:28 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:53)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:342)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:81)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:669)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1296)
	at o

In [24]:
df.schema.names

['id_transaction',
 'client_nom',
 'client_age',
 'client_ville',
 'produit_nom',
 'produit_categorie',
 'produit_marque',
 'prix_catalogue',
 'magasin_nom',
 'magasin_type',
 'magasin_region',
 'date',
 'quantite',
 'purchase_date',
 'montant_total']